In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
pip install PyPDF2 sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.2 MB/s eta 0:00:00


In [26]:
import PyPDF2
import os

def extract_text_from_pdfs(pdf_folder):
    pdf_texts = {}

    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''
                for page in reader.pages:
                    text += page.extract_text() + '\n'
                pdf_texts[filename] = text.strip()  # Store the text with the filename as the key

    return pdf_texts

In [27]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

def create_embeddings(texts):
    # Create embeddings for each text entry
    embeddings = model.encode(list(texts.values()), convert_to_numpy=True)
    return embeddings


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
import faiss

def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings
    index = faiss.IndexFlatL2(dimension)  # Create a flat L2 index
    index.add(embeddings)  # Add embeddings to the index
    return index


In [29]:
def retrieve_similar_documents(query, model, index, texts, top_k=5):
    # Create an embedding for the query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Perform the FAISS search
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the documents based on indices
    similar_docs = {list(texts.keys())[i]: texts[list(texts.keys())[i]] for i in indices[0]}

    return similar_docs


In [34]:
import PyPDF2
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Step 1: Extract Text from a Single PDF
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text.strip()  # Return the extracted text

# Step 2: Create Embeddings
def create_embeddings(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Load the model
    embedding = model.encode([text], convert_to_numpy=True)  # Create embedding for the text
    return embedding

# Step 3: Store Embeddings with FAISS
def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings
    index = faiss.IndexFlatL2(dimension)  # Create a flat L2 index
    index.add(embeddings)  # Add the embeddings to the index
    return index

# Step 4: Query and Retrieve Relevant Text
def retrieve_similar_documents(query, model, index, text):
    query_embedding = model.encode([query], convert_to_numpy=True)  # Create an embedding for the query
    distances, indices = index.search(query_embedding, 1)  # Search for the nearest embedding
    return text  # Return the original text

# Putting it all together
def main():
    # Step 1: Extract text from a single PDF
    pdf_file = '/content/drive/MyDrive/Apple-ProfessionalDevelopmentCatalogWeb.pdf'  # Path to your PDF file
    pdf_text = extract_text_from_pdf(pdf_file)

    # Step 2: Create embeddings for the extracted text
    embeddings = create_embeddings(pdf_text)

    # Step 3: Build the FAISS index
    index = build_faiss_index(embeddings)

    print("Welcome to the Product Recommendation Insights Bot!")
    print("Ask your question, or type 'exit' to quit.")

    while True:
        query = input("\nYour Question: ")
        if query.lower() == 'exit':
            break

        # Step 4: Retrieve relevant documents based on the query
        similar_document = retrieve_similar_documents(query, model, index, pdf_text)

        # Display result
        print(f"\nRelevant Information:\n{similar_document[:500]}...")  # Display the first 500 characters of the content

if __name__ == "__main__":
    main()


Welcome to the Product Recommendation Insights Bot!
Ask your question, or type 'exit' to quit.

Your Question: What are the ios devices

Relevant Information:
Apple Professional  Development Catalogue December 2014 

Leadership | Foundations | Curriculum | Support | Preschool Table of Contents Get the most from your Apple products 2 ..........................................Leadership 3 ........................................................................................Education Strategic Planning 3 ................................................................Vision and Plan (iOS) 4 .................................................................

Your Question: List the ios basics

Relevant Information:
Apple Professional  Development Catalogue December 2014 

Leadership | Foundations | Curriculum | Support | Preschool Table of Contents Get the most from your Apple products 2 ..........................................Leadership 3 ...............................................